<a href="https://colab.research.google.com/github/chloepar/Stack_Overflow_Survey_Exploration_DSBA6211/blob/main/6211_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests, zipfile, io, time, os, logging

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36'
}

### Load surveys for 2011 - 2015, 2017 - 2025 (2016 has a different folder format)

In [14]:

def find_csv_in_dir(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                return os.path.join(root, file)
    raise FileNotFoundError(f"No CSV found in {directory}")

# Define available years
available_years = range(2011, 2026)
dfs = {}

# Retry settings
max_retries = 10
base_delay = 30  # seconds

for year in available_years:
    url = f'https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-{year}.zip'
    extract_path = f'/content/survey_{year}'
    print(f"\n📦 Processing {year}...")

    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(url)
            content_type = r.headers.get('Content-Type', '')

            if r.status_code == 429:
                retry_after = int(r.headers.get('Retry-After', base_delay))
                print(f"⏳ Rate limited. Waiting {retry_after} seconds before retrying...")
                time.sleep(retry_after)
                continue

            if r.status_code == 200 and 'zip' in content_type:
                z = zipfile.ZipFile(io.BytesIO(r.content))
                z.extractall(extract_path)

                csv_path = find_csv_in_dir(extract_path)
                df = pd.read_csv(csv_path, encoding='ISO-8859-1')
                df['year'] = year  # Add the year column
                dfs[year] = df
                print(f"✅ Loaded {year} survey with {len(df)} rows.")
                break  # success, exit retry loop
            else:
                print(f"❌ Failed for {year}: Status {r.status_code}, Content-Type {content_type}")
                break  # don't retry non-429 failures

        except zipfile.BadZipFile:
            print(f"⚠️ Invalid ZIP file for {year}. Attempt {attempt}/{max_retries}")
        except Exception as e:
            print(f"⚠️ Error on attempt {attempt}/{max_retries} for {year}: {e}")

        # Exponential backoff before next retry
        wait = base_delay * (2 ** (attempt - 1))
        print(f"🔁 Retrying in {wait} seconds...")
        time.sleep(wait)



📦 Processing 2011...
✅ Loaded 2011 survey with 2814 rows.

📦 Processing 2012...
✅ Loaded 2012 survey with 6244 rows.

📦 Processing 2013...


/tmp/ipython-input-2658173456.py:37: DtypeWarning: Columns (46,48,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='ISO-8859-1')


✅ Loaded 2013 survey with 9743 rows.

📦 Processing 2014...
✅ Loaded 2014 survey with 7644 rows.

📦 Processing 2015...


/tmp/ipython-input-2658173456.py:37: DtypeWarning: Columns (5,108,121,196,197,198) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='ISO-8859-1')


✅ Loaded 2015 survey with 26087 rows.

📦 Processing 2016...
✅ Loaded 2016 survey with 0 rows.

📦 Processing 2017...
✅ Loaded 2017 survey with 51392 rows.

📦 Processing 2018...
⏳ Rate limited. Waiting 30 seconds before retrying...
⏳ Rate limited. Waiting 30 seconds before retrying...


/tmp/ipython-input-2658173456.py:37: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='ISO-8859-1')


✅ Loaded 2018 survey with 98855 rows.

📦 Processing 2019...
✅ Loaded 2019 survey with 88883 rows.

📦 Processing 2020...
✅ Loaded 2020 survey with 64461 rows.

📦 Processing 2021...
⏳ Rate limited. Waiting 30 seconds before retrying...
⏳ Rate limited. Waiting 30 seconds before retrying...
✅ Loaded 2021 survey with 83439 rows.

📦 Processing 2022...
✅ Loaded 2022 survey with 73268 rows.

📦 Processing 2023...
⏳ Rate limited. Waiting 30 seconds before retrying...
⏳ Rate limited. Waiting 30 seconds before retrying...
✅ Loaded 2023 survey with 89184 rows.

📦 Processing 2024...
✅ Loaded 2024 survey with 65437 rows.

📦 Processing 2025...
⏳ Rate limited. Waiting 30 seconds before retrying...
⏳ Rate limited. Waiting 30 seconds before retrying...
✅ Loaded 2025 survey with 49123 rows.


### Extract 2016

In [15]:
# Setup logging
logging.basicConfig(level=logging.INFO)

def download_and_extract_zip(url, headers, extract_path, max_retries=3, delay=5):
    for attempt in range(1, max_retries + 1):
        try:
            logging.info(f"Attempt {attempt}: Downloading {url}")
            response = requests.get(url, headers=headers, timeout=30)
            response.raise_for_status()  # Raise HTTPError for bad responses
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                z.extractall(extract_path)
            logging.info("Download and extraction successful.")
            return True
        except (requests.RequestException, zipfile.BadZipFile) as e:
            logging.warning(f"Attempt {attempt} failed: {e}")
            if attempt < max_retries:
                time.sleep(delay)
            else:
                logging.error("All attempts failed.")
                return False

# Parameters
url_2016 = 'https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-2016.zip'
extract_path_2016 = '/content/survey_2016'

# Run download
success = download_and_extract_zip(url_2016, headers, extract_path_2016)

# Load CSV if successful
if success:
    csv_path = '/content/survey_2016/2016 Stack Overflow Survey Results/2016 Stack Overflow Survey Responses.csv'
    try:
        df_2016 = pd.read_csv(csv_path, encoding='ISO-8859-1')
        df_2016['year'] = '2016'  # Add the year column
        logging.info("CSV loaded successfully.")
    except Exception as e:
        logging.error(f"Failed to load CSV: {e}")
else:
    df_2016 = None


### Extract dataframes for each year

In [16]:
for year, df in dfs.items():
    globals()[f'df_{year}'] = df

### Combine all dataframes from 2011 to 2025


In [ ]:
all_dfs = list(dfs.values()) + [df_2016]
df = pd.concat(all_dfs, ignore_index=True)
display(df.head())

In [ ]:
all_dfs.describe(include='all').T

In [ ]:
# Identify columns in each DataFrame for all available years
all_years = list(dfs.keys()) + [2016] # Include 2016 in the list of years
cols_by_year = {year: set(dfs[year].columns) if year != 2016 else set(df_2016.columns) for year in all_years}

# Get all unique columns across all years
all_unique_cols = set().union(*cols_by_year.values())

# Create a dictionary to store column presence in each year
column_presence = {year: [col in cols_by_year[year] for col in all_unique_cols] for year in all_years}

# Create a pandas DataFrame (matrix) from the dictionary
column_matrix = pd.DataFrame(column_presence, index=list(all_unique_cols))

# Add a column to count the number of dataframes the column is in
column_matrix['PresenceCount'] = column_matrix.sum(axis=1)

# Sort the matrix by 'PresenceCount' in descending order
column_matrix_sorted = column_matrix.sort_values(by='PresenceCount', ascending=False)

# Drop the 'PresenceCount' column for the final display
column_matrix_sorted = column_matrix_sorted.drop(columns=['PresenceCount'])

# Display the sorted matrix
display(column_matrix_sorted)

In [ ]:
# Check for missing values in 'Country' column
print("Missing values in 'Country' column:")
print(df['Country'].isnull().sum())

# Drop rows with missing 'Country' values
df_country = df.dropna(subset=['Country']).copy()

# Group by 'Country' and 'Year' and count occurrences
country_counts = df_country.groupby(['Country', 'Year']).size().reset_index(name='Count')

# Display the counts
display(country_counts.head())

# Pivot the table for easier plotting
country_pivot = country_counts.pivot(index='Country', columns='Year', values='Count').fillna(0)

# Display the pivoted table
display(country_pivot.head())

# Optional: Visualize the top N countries over the years
top_n = 10
top_countries = country_pivot.sum(axis=1).nlargest(top_n).index
country_pivot_top = country_pivot.loc[top_countries]

# Reset index to make 'Country' a column and melt for plotting
country_pivot_top_reset = country_pivot_top.reset_index()
country_melted = country_pivot_top_reset.melt(id_vars='Country', var_name='Year', value_name='Count')

plt.figure(figsize=(15, 7))
sns.lineplot(data=country_melted, x='Year', y='Count', hue='Country')

plt.title(f'Top {top_n} Stack Overflow Users by Country and Year')
plt.xlabel('Year')
plt.ylabel('Number of Users')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()